In [1]:
import pandas as pd

REVIEWS_POLARITY_FILE = 'reviews_polarity.jsonlines'

reviews = pd.read_json(REVIEWS_POLARITY_FILE, lines=True)

In [2]:
reviews.head()

,type,review,reviewTitle,rating,datePublished,book_id,review_id,lang,polarity
0,review,"Добралася до відомого роману, який нещодавно в...",Сердечна книга!,5,2016-06-1,0,0,uk,POSITIVE
1,review,История тут о замужней женщине - Элизабет. Ист...,Очень нудно…Но не без смысла.,2,2017-01-3,0,1,ru,
2,review,"Очень рада, что столь известный роман Элизабет...",Лекарство от душевной тоски,5,2016-08-5,0,2,ru,
3,review,"Эту книгу я решила прочитать, потому что все в...",Скучно и неинтересно,2,2017-06-28,0,3,ru,
4,review,"Книга Елізабет Гілберт ""Їсти. Молитися. Кохати...","""Їсти. Молитися. Кохати""",4,2018-08-15,0,4,uk,POSITIVE


In [3]:
polarity_reviews = reviews[(reviews['polarity'] != "")]

In [4]:
book_id_polarity_df = polarity_reviews.groupby(['book_id', 'polarity']).size().unstack(fill_value=0)

In [5]:
def calculate_polarity_rating(row):
    positive = row['POSITIVE']
    negative = row['NEGATIVE']
    sum = float(positive + negative)
    rating = (5.0 / sum) * positive
    row['polarity_rating'] = rating
    return row.round(2)

polarity_rating_df = book_id_polarity_df.apply(calculate_polarity_rating, axis=1)

In [6]:
# Add rating

BOOKS_FILE = 'books.jsonlines'
books = pd.read_json(BOOKS_FILE, lines=True)

book_id_book = books.set_index('book_id')
books_polarity_rating_df = book_id_book.join(polarity_rating_df, on="book_id")
books_polarity_rating_df = books_polarity_rating_df.reset_index()

In [7]:
books_polarity_rating_json_lines = books_polarity_rating_df.to_json(orient='records', lines=True, force_ascii=False)

In [8]:
OUTPUT_FILE_PATH = "./books_polarity_rating.jsonlines"

with open(OUTPUT_FILE_PATH, "w") as f:
    f.write(books_polarity_rating_json_lines)